In [1]:
# Dependencies
import pandas as pd

In [2]:
# import datasets
disasters_by_yr = '../resources/natural_disaster_duration.csv'
gnrl_df = '../resources/general_dataset.csv' 

count_disaster = pd.read_csv(disasters_by_yr)
full_df = pd.read_csv(gnrl_df)

In [3]:
# Setting option in pandas to view all columns
pd.set_option('max_columns', None)

In [4]:
# Displaying first 3 rows
count_disaster.head(3)

,dis_no,year,seq,disaster_group,disaster_subgroup,disaster_type,disaster_subtype,event_name,country,country_code,region,continent,location,dis_mag_scale,start_year,start_month,start_day,end_year,end_month,end_day,total_deaths,total_affected,total_damages_('000 US$),total_damages_adjusted_('000 US$),cpi
0,1970-0013-ARG,1970,13,Natural,Hydrological,Flood,NaN,NaN,Argentina,ARG,South America,Americas,Mendoza,Km2,1970,1.0,4.0,1970,1.0,4.0,36.0,NaN,25000.0,174481.0,14.32817
1,1970-0109-AUS,1970,109,Natural,Meteorological,Storm,Tropical cyclone,Ada,Australia,AUS,Australia and New Zealand,Oceania,Queensland,Kph,1970,1.0,NaN,1970,1.0,NaN,13.0,NaN,72475.0,505822.0,14.32817
2,1970-0044-BEN,1970,44,Natural,Hydrological,Flood,NaN,NaN,Benin,BEN,Western Africa,Africa,Atacora region,Km2,1970,9.0,NaN,1970,9.0,NaN,NaN,NaN,200.0,1396.0,14.32817


In [5]:
#list of all columns in count df
count_disaster.columns.to_list()

['dis_no',
 'year',
 'seq',
 'disaster_group',
 'disaster_subgroup',
 'disaster_type',
 'disaster_subtype',
 'event_name',
 'country',
 'country_code',
 'region',
 'continent',
 'location',
 'dis_mag_scale',
 'start_year',
 'start_month',
 'start_day',
 'end_year',
 'end_month',
 'end_day',
 'total_deaths',
 'total_affected',
 "total_damages_('000 US$)",
 "total_damages_adjusted_('000 US$)",
 'cpi']

In [6]:
# Drop columns that are not required. Columns needed: 'year','country_code','disaster_type', 'start_year'
count_dis_c = count_disaster[['year','country_code','disaster_type', 'start_year']]

In [7]:
# Displaying the unique disasters to filter the ones that are needed
count_disaster['disaster_type'].unique()

array(['Flood', 'Storm', 'Drought', 'Landslide', 'Earthquake', 'Epidemic',
       'Wildfire', 'Mass movement (dry)', 'Extreme temperature',
       'Volcanic activity', 'Insect infestation', 'Impact',
       'Animal accident', 'Glacial lake outburst'], dtype=object)

In [45]:
count_dis_c.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15012 entries, 0 to 15011
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   year           15012 non-null  int64 
 1   country_code   15012 non-null  object
 2   disaster_type  15012 non-null  object
 3   start_year     15012 non-null  int64 
dtypes: int64(2), object(2)
memory usage: 469.2+ KB


In [9]:
# Grouping by year, country code and disaster to get the number of disasters by the grouping
count_disc_c_gb = count_dis_c.groupby(['year','country_code','disaster_type']).count()
count_disc_c_gb.head()

start_year
year country_code disaster_type            
1970 ARG          Flood                   1
     AUS          Storm                   1
     BEN          Flood                   1
     BGD          Flood                   1
                  Storm                   3

In [10]:
# Resetting the index
count = count_disc_c_gb.reset_index()
count.head(3)

,year,country_code,disaster_type,start_year
0,1970,ARG,Flood,1
1,1970,AUS,Storm,1
2,1970,BEN,Flood,1


In [11]:
# Filtering by 1982 and France to validate grouping was done correctly and gives us the information required
count_dis_c[(count_dis_c['year'] == 1982) & (count_dis_c['country_code'] == 'FRA') ]

,year,country_code,disaster_type,start_year
851,1982,FRA,Drought,1982
886,1982,FRA,Landslide,1982
887,1982,FRA,Landslide,1982
888,1982,FRA,Wildfire,1982
889,1982,FRA,Storm,1982
890,1982,FRA,Storm,1982


In [12]:
# Validating with the above filter the data calculated by the grouping is correct
count[(count['year'] == 1982) & (count['country_code'] == 'FRA')]

,year,country_code,disaster_type,start_year
908,1982,FRA,Drought,1
909,1982,FRA,Landslide,2
910,1982,FRA,Storm,2
911,1982,FRA,Wildfire,1


In [13]:
# Filtering the df to only include the disasters linked to temperature anomalies
disaster_types = ['Drought', 'Extreme temperature', 'Flood', 'Storm','Wildfire']
count_c = count[count.disaster_type.isin(disaster_types)]

In [14]:
# Getting the unique of the df without the filter
count['disaster_type'].unique()

array(['Flood', 'Storm', 'Drought', 'Landslide', 'Earthquake', 'Epidemic',
       'Wildfire', 'Mass movement (dry)', 'Extreme temperature',
       'Volcanic activity', 'Insect infestation', 'Impact',
       'Animal accident', 'Glacial lake outburst'], dtype=object)

In [15]:
# Shape of df with filter of disasters
count_c.shape

(6401, 4)

In [16]:
# Validating disaster
count_c['disaster_type'].unique()

array(['Flood', 'Storm', 'Drought', 'Wildfire', 'Extreme temperature'],
      dtype=object)

In [17]:
count_c_long_to_wide = count_c.pivot_table(values='start_year', index= ['year','country_code'], columns='disaster_type')
count_c_long_to_wide.head(3)

disaster_type      Drought  Extreme temperature  Flood  Storm  Wildfire
year country_code                                                      
1970 ARG               NaN                  NaN    1.0    NaN       NaN
     AUS               NaN                  NaN    NaN    1.0       NaN
     BEN               NaN                  NaN    1.0    NaN       NaN

In [18]:
count_c_long_to_wide=count_c_long_to_wide.fillna(0)

In [19]:
count_c_long_to_wide.head(3)

disaster_type      Drought  Extreme temperature  Flood  Storm  Wildfire
year country_code                                                      
1970 ARG               0.0                  0.0    1.0    0.0       0.0
     AUS               0.0                  0.0    0.0    1.0       0.0
     BEN               0.0                  0.0    1.0    0.0       0.0

In [20]:
# Adding the total number of disasters by year and country code
count_c_long_to_wide['total_count_disasters'] = count_c_long_to_wide['Drought']+count_c_long_to_wide['Extreme temperature']+count_c_long_to_wide['Flood']+count_c_long_to_wide['Storm']+count_c_long_to_wide['Wildfire']

In [21]:
count_c_long_to_wide.head(3)

disaster_type      Drought  Extreme temperature  Flood  Storm  Wildfire  \
year country_code                                                         
1970 ARG               0.0                  0.0    1.0    0.0       0.0   
     AUS               0.0                  0.0    0.0    1.0       0.0   
     BEN               0.0                  0.0    1.0    0.0       0.0   

disaster_type      total_count_disasters  
year country_code                         
1970 ARG                             1.0  
     AUS                             1.0  
     BEN                             1.0

In [22]:
# Rename columns
count_c_long_to_wide.rename(columns = {'Drought':'drought_count', 'Extreme temperature' : 'extreme_temperatures_count', 'Flood':'floods_count','Storm':'storms_count','Wildfire':'wildfires_count'}, inplace=True)
count_c_long_to_wide.head(3)

disaster_type      drought_count  extreme_temperatures_count  floods_count  \
year country_code                                                            
1970 ARG                     0.0                         0.0           1.0   
     AUS                     0.0                         0.0           0.0   
     BEN                     0.0                         0.0           1.0   

disaster_type      storms_count  wildfires_count  total_count_disasters  
year country_code                                                        
1970 ARG                    0.0              0.0                    1.0  
     AUS                    1.0              0.0                    1.0  
     BEN                    0.0              0.0                    1.0

In [24]:
#Merge of full df with count df
full_new = full_df.merge(count_c_long_to_wide, on=['country_code','year'])

In [25]:
full_new.head(3)

,country_code,country_name,year,agriculture_value_added_constants,agriculture_value_added_percentage_gdp,annual_co2_emissions_tons,inflation_inflation_gdp_deflation,deaths_from_drought,deaths_from_extreme_temps,deaths_from_floods,deaths_from_storms,deaths_from_wildfires,people_affected_by_drought_per_100k,people_affected_by_extreme_temperatures_per_100k,people_affected_by_floods_per_100k,people_affected_by_storms_per_100k,people_affected_by_wildfires_per_100k,economic_damages_from_drought,economic_damages_from_drought_as_a_share_of_gdp,economic_damages_from_extreme_temperatures,economic_damages_from_extreme_temperatures_as_a_share_of_gdp,economic_damages_from_floods,economic_damages_from_floods_as_a_share_of_gdp,economic_damages_from_storms,economic_damages_from_storms_as_a_share_of_gdp,economic_damages_from_wildfires,economic_damages_as_a_share_of_gdp,economic_damages,economic_damages_thousands,people_affected_per_100k,number_of_deaths,population_population_growth_annual_percentage,population_population_total,temperature_change_temp_change_celsius,drought_count,extreme_temperatures_count,floods_count,storms_count,wildfires_count,total_count_disasters
0,AFG,Afghanistan,1971,NaN,NaN,1893554,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,2.665129,11475450,0.652,1.0,0.0,0.0,0.0,0.0,1.0
1,AFG,Afghanistan,1972,NaN,NaN,1530347,NaN,0.0,0.0,150.0,0.0,0.0,0.0,0.0,2120.261216,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2120.261216,150.0,2.714539,11791222,-1.089,0.0,0.0,1.0,0.0,0.0,1.0
2,AFG,Afghanistan,1976,NaN,NaN,1980859,NaN,0.0,0.0,51.0,0.0,0.0,0.0,0.0,618.094723,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,618.094723,51.0,1.981389,12943093,-0.295,0.0,0.0,1.0,0.0,0.0,1.0


In [26]:
# Filter data based on years being analyzed 1988-2020
full_new_f = full_new[full_new['year'] >=1988] 

In [27]:
# Validating that the data start at year 1988
full_new_f['year'].min()

1988

In [28]:
# Viewing the columns as a list 
full_new_f.columns.to_list()

['country_code',
 'country_name',
 'year',
 'agriculture_value_added_constants',
 'agriculture_value_added_percentage_gdp',
 'annual_co2_emissions_tons',
 'inflation_inflation_gdp_deflation',
 'deaths_from_drought',
 'deaths_from_extreme_temps',
 'deaths_from_floods',
 'deaths_from_storms',
 'deaths_from_wildfires',
 'people_affected_by_drought_per_100k',
 'people_affected_by_extreme_temperatures_per_100k',
 'people_affected_by_floods_per_100k',
 'people_affected_by_storms_per_100k',
 'people_affected_by_wildfires_per_100k',
 'economic_damages_from_drought',
 'economic_damages_from_drought_as_a_share_of_gdp',
 'economic_damages_from_extreme_temperatures',
 'economic_damages_from_extreme_temperatures_as_a_share_of_gdp',
 'economic_damages_from_floods',
 'economic_damages_from_floods_as_a_share_of_gdp',
 'economic_damages_from_storms',
 'economic_damages_from_storms_as_a_share_of_gdp',
 'economic_damages_from_wildfires',
 'economic_damages_as_a_share_of_gdp',
 'economic_damages',
 'econo

In [29]:
# Re order columns
full_new_o = full_new_f.reindex(columns = ['country_code', 'country_name', 'year', 'annual_co2_emissions_tons', 'inflation_inflation_gdp_deflation', 'deaths_from_drought', 'deaths_from_extreme_temps', 'deaths_from_floods', 'deaths_from_storms', 'deaths_from_wildfires', 'people_affected_by_drought_per_100k', 'people_affected_by_extreme_temperatures_per_100k', 'people_affected_by_floods_per_100k', 'people_affected_by_storms_per_100k', 'people_affected_by_wildfires_per_100k', 'economic_damages_from_drought', 'economic_damages_from_drought_as_a_share_of_gdp', 'economic_damages_from_extreme_temperatures', 'economic_damages_from_extreme_temperatures_as_a_share_of_gdp', 'economic_damages_from_floods', 'economic_damages_from_floods_as_a_share_of_gdp', 'economic_damages_from_storms', 'economic_damages_from_storms_as_a_share_of_gdp', 'economic_damages_from_wildfires', 'economic_damages_as_a_share_of_gdp', 'economic_damages', 'economic_damages_thousands', 'people_affected_per_100k', 'number_of_deaths', 'drought_count', 'extreme_temperatures_count', 'floods_count', 'storms_count', 'wildfires_count', 'total_count_disasters', 'population_population_growth_annual_percentage', 'population_population_total', 'temperature_change_temp_change_celsius', 'agriculture_value_added_constants', 'agriculture_value_added_percentage_gdp'])

### Removing duplicated rows from general data set

In [30]:
# Drop duplicates based on year and country code.
full_new_o = full_new_o.drop_duplicates(subset=['country_code','year'], keep='first')

In [31]:
# Visualizing the first 3 rows of data
full_new_o.head(3)

,country_code,country_name,year,annual_co2_emissions_tons,inflation_inflation_gdp_deflation,deaths_from_drought,deaths_from_extreme_temps,deaths_from_floods,deaths_from_storms,deaths_from_wildfires,people_affected_by_drought_per_100k,people_affected_by_extreme_temperatures_per_100k,people_affected_by_floods_per_100k,people_affected_by_storms_per_100k,people_affected_by_wildfires_per_100k,economic_damages_from_drought,economic_damages_from_drought_as_a_share_of_gdp,economic_damages_from_extreme_temperatures,economic_damages_from_extreme_temperatures_as_a_share_of_gdp,economic_damages_from_floods,economic_damages_from_floods_as_a_share_of_gdp,economic_damages_from_storms,economic_damages_from_storms_as_a_share_of_gdp,economic_damages_from_wildfires,economic_damages_as_a_share_of_gdp,economic_damages,economic_damages_thousands,people_affected_per_100k,number_of_deaths,drought_count,extreme_temperatures_count,floods_count,storms_count,wildfires_count,total_count_disasters,population_population_growth_annual_percentage,population_population_total,temperature_change_temp_change_celsius,agriculture_value_added_constants,agriculture_value_added_percentage_gdp
5,AFG,Afghanistan,1988,2856896,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,1385.780685,0.0,0.0,0.0,0.0,0.0,0.0,260000.0,0.0,0.0,0.0,0.0,0.0,260000.0,260000.0,1385.780685,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.116008,11618008,0.988,NaN,NaN
6,AFG,Afghanistan,1990,2603238,NaN,0.0,144.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,144.0,0.0,1.0,0.0,0.0,0.0,1.0,4.476954,12412311,0.847,NaN,NaN
7,AFG,Afghanistan,1991,2427366,NaN,0.0,80.0,1193.0,0.0,0.0,0.0,1.503872,977.517107,0.0,0.0,0.0,0.0,0.0,0.0,60000.0,0.0,0.0,0.0,0.0,0.0,60000.0,60000.0,977.517107,1273.0,0.0,1.0,3.0,0.0,0.0,4.0,6.900124,13299016,-0.058,NaN,NaN


In [32]:
test = full_new_o[['year','country_code','country_name', 'total_count_disasters']]

In [33]:
chn = test[test['country_code'] == 'CHN']
chn.groupby('year')['total_count_disasters'].sum()

year
1988    13.0
1989     8.0
1990    14.0
1991     9.0
1992    16.0
1993    15.0
1994    14.0
1995    11.0
1996    12.0
1997    15.0
1998    10.0
1999    17.0
2000    22.0
2001    21.0
2002    20.0
2003    14.0
2004    17.0
2005    28.0
2006    29.0
2007    17.0
2008    19.0
2009    19.0
2010    13.0
2011    11.0
2012    21.0
2013    31.0
2014    31.0
2015    30.0
2016    27.0
2017    22.0
2018    20.0
2019    11.0
2020    10.0
Name: total_count_disasters, dtype: float64

In [34]:
# Generating csv of final general data set without the index
full_new_o.to_csv('../resources/general_dataset_v5.csv', index=False)

In [35]:
# Making a copy of the final general data set to create pivot tables
full_filter = full_new_o.copy()

In [36]:
# Pivot table to get the top countries by the number of disasters from 1988 to 2020
piv = full_filter.pivot_table(values='total_count_disasters', index=['country_code','country_name'], aggfunc=sum).sort_values(by='total_count_disasters', ascending=False)

In [37]:
# Visualizing the top countries by the sum of total disasters
piv.head(10)

,,total_count_disasters
country_code,country_name,
USA,United States,765.0
CHN,China,587.0
IND,India,392.0
PHL,Philippines,367.0
BGD,Bangladesh,209.0
IDN,Indonesia,208.0
VNM,Vietnam,197.0
MEX,Mexico,174.0
AUS,Australia,160.0


In [40]:
# Top 10 countries based on the average of annual_co2_emissions_tons
piv2 = full_filter.pivot_table(values='annual_co2_emissions_tons', index=['country_code','country_name']).sort_values(by='annual_co2_emissions_tons', ascending=False)
piv2.head(10)

,,annual_co2_emissions_tons
country_code,country_name,
CHN,China,6.058611e+09
USA,United States,5.540071e+09
RUS,Russia,1.608541e+09
IND,India,1.363983e+09
JPN,Japan,1.211410e+09
DEU,Germany,8.689846e+08
CAN,Canada,5.390040e+08
GBR,United Kingdom,5.218149e+08
SAU,Saudi Arabia,5.163637e+08


In [42]:
# Top 10 countries based on the average temperature_change_temp_change_celsius
piv3 = full_filter.pivot_table(values='temperature_change_temp_change_celsius', index=['country_code','country_name']).sort_values(by='temperature_change_temp_change_celsius', ascending=False)
piv3.head(10)

,,temperature_change_temp_change_celsius
country_code,country_name,
QAT,Qatar,1.981000
SVN,Slovenia,1.813833
EST,Estonia,1.800667
SWE,Sweden,1.732857
FIN,Finland,1.720500
SRB,Serbia,1.717636
CPV,Cape Verde,1.670000
ARE,United Arab Emirates,1.666000
ARM,Armenia,1.529000


In [43]:
# Top 10 countries based on the average economic_damages_as_a_share_of_gdp
piv4 = full_filter.pivot_table(values='economic_damages_as_a_share_of_gdp', index=['country_code','country_name']).sort_values(by='economic_damages_as_a_share_of_gdp', ascending=False)
piv4.head(10)

,,economic_damages_as_a_share_of_gdp
country_code,country_name,
DMA,Dominica,16.284173
WSM,Samoa,11.417148
GRD,Grenada,9.345356
KNA,Saint Kitts and Nevis,8.470519
TCA,Turks and Caicos Islands,6.679056
ATG,Antigua and Barbuda,3.460213
GUY,Guyana,1.694616
BLZ,Belize,1.358093
VUT,Vanuatu,1.156070


In [44]:
# Top 10 countries based on the average people_affected_per_100k
piv5 = full_filter.pivot_table(values='people_affected_per_100k', index=['country_code','country_name']).sort_values(by='people_affected_per_100k', ascending=False)
piv5.head(10)

,,people_affected_per_100k
country_code,country_name,
CHN,China,704944.463120
USA,United States,277296.471861
FRA,France,156937.351199
ITA,Italy,156301.863386
DEU,Germany,67265.711102
CAN,Canada,66709.070171
ESP,Spain,59300.092182
JOR,Jordan,58140.624780
RUS,Russia,53940.884161
